In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imageio
import os

class Cell:
    def __init__(self, x, y, theta, length=1.0):
        self.x = x
        self.y = y
        self.theta = theta
        self.length = length

    def move(self, speed, dt):
        self.x += speed * np.cos(self.theta) * dt
        self.y += speed * np.sin(self.theta) * dt

    def rotate(self, angular_velocity, dt):
        self.theta += angular_velocity * dt


def generate_random_point_in_circle(radius):
    """
    Generate a random point (x,y) within a circle of given radius.
    """
    r = radius * np.sqrt(np.random.uniform(0, 1))
    theta = np.random.uniform(0, 2*np.pi)
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y


        
def calculate_density(x, y, cells, radius=2.0):
    count = 0
    for cell in cells:
        distance = np.sqrt((cell.x - x)**2 + (cell.y - y)**2)
        if distance < radius:
            count += 1
    return count

def average_angle(x, y, cells, radius=2.0):
    total_sin = 0
    total_cos = 0
    count = 0
    for cell in cells:
        distance = np.sqrt((cell.x - x)**2 + (cell.y - y)**2)
        if distance < radius:
            total_sin += np.sin(cell.theta)
            total_cos += np.cos(cell.theta)
            count += 1
    if count == 0:
        return np.random.uniform(0, 2*np.pi)
    avg_angle = np.arctan2(total_sin / count, total_cos / count)
    return avg_angle

def get_angle_difference(a, b):
    """Calculate the difference between two angles in the range [-pi, pi]"""
    diff = a - b
    while diff > np.pi:
        diff -= 2 * np.pi
    while diff < -np.pi:
        diff += 2 * np.pi
    return diff

def simulate(cells, num_steps, base_speed=5, dt=0.1, angular_velocity=0.1):
    filenames = []
    density_threshold = 20  # Density threshold for speed reduction
    alignment_threshold = 40  # Density threshold for alignment
    reduced_speed = 4  # Speed when density exceeds the threshold
    alignment_strength = 0.07  # Control alignment behavior
    
    for step in range(num_steps):
        # Cell division at every 10th step
        if step % 500 == 0:
            new_cells = [Cell(cell.x, cell.y, cell.theta) for cell in cells]  # New cells have same direction as mother cell
            cells.extend(new_cells)

        for cell in cells:
            density = calculate_density(cell.x, cell.y, cells)
            if density > alignment_threshold:
                avg_angle = average_angle(cell.x, cell.y, cells)
                angle_diff = get_angle_difference(avg_angle, cell.theta)
                adjustment = alignment_strength * angle_diff
                cell.theta = (cell.theta + adjustment) % (2 * np.pi)
            
            speed = base_speed if density < density_threshold else reduced_speed
            cell.move(speed, dt)
            cell.rotate(angular_velocity, dt)
            
        filename = f"step_{step}.png"
        filenames.append(filename)
        visualize(cells, filename)
    
    # Create a gif
    images = [imageio.imread(filename) for filename in filenames]
    imageio.mimsave('simulation.gif', images, duration=0.5)
    for filename in filenames:
        os.remove(filename)  # Clean up the individual step images



def visualize(cells, filename):
    fig, ax = plt.subplots()
    for cell in cells:
        rect = patches.Rectangle(
            (cell.x - cell.length/2*np.cos(cell.theta), cell.y - cell.length/2*np.sin(cell.theta)),
            cell.length, 0.1, angle=np.degrees(cell.theta), color='blue'
        )
        ax.add_patch(rect)
    ax.set_xlim(-120, 120)
    ax.set_ylim(-120, 120)
    plt.savefig(filename)
    plt.close()



# Create initial seeds
num_seeds = 1
cells_per_seed = 400
cells = []
seed_radius = 40  # The seeds will be placed in a larger circle to ensure they don't overlap
cell_radius = 2   # The individual cells around the seed will be placed in a smaller circle

for _ in range(num_seeds):
    seed_x, seed_y = generate_random_point_in_circle(seed_radius) # Placing seeds
    for _ in range(cells_per_seed):
        offset_x, offset_y = generate_random_point_in_circle(cell_radius) # Generate cells around the seed position
        x = seed_x + offset_x
        y = seed_y + offset_y
        angle = np.random.uniform(0, 2*np.pi)
        cells.append(Cell(x, y, angle))

simulate(cells, num_steps=150)


/var/folders/mx/811ntzld4v34dwbtw7y559dw0000gq/T/ipykernel_50490/2822627093.py:96: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(filename) for filename in filenames]
